In [ ]:
# this notebook demonstrate how to cuate dataset with the utitlity function provided
# it involved two steps, first we create or train appropriate agent, second is to sample those agents interaction with the environment and store them in a dataset

# Let's get the stock ticker we would use for curating the dataset

# this library is to get the list of tickers from NASDAQ and DOWJONES
from gettickers import *
nasdaq_tic = get_nasdaq_tickers()
dow_tic = get_dow_tickers()

# merge the lists and remove duplicates
all_tickers = nasdaq_tic + dow_tic
all_tickers = list(set(all_tickers))

# let's get the first 10 tickers and check the number of tickers in the lists
print(all_tickers[:10])
print(len(nasdaq_tic))
print(len(dow_tic))
print(len(all_tickers))

# agents can be created from RL (stable-baselines3)
# set date range for stock data to train RL-agents

# this will set  the parameter for the training of the agents
start_date = "2017-01-01"
num_days = 400
interval = "1d"
indicators = ["Volume", "volume_cmf", "trend_macd", "momentum_rsi", "momentum_stoch_rsi", "trend_sma_fast"]
exlucde = ["positive", "negative", "neutral"]
init_balance = 20000
agent_output_path = "trained_stable_agents/"
stable_training_json_path = "training_stable_agents_config/"
normalize_params = {}

# save these parameter as json config file using the utility function from getsttickers
#create_json_files(all_tickers, start_date, num_days, interval, indicators, init_balance, agent_output_path, stable_training_json_path, normalize_params)

In [ ]:
# we want to train our agents with a normalize environment, therefore we need to calculate the mean and std of the stock trading data
# we can do this by sampling other agents and storing their interaction with the environment in a dataset
# then we can use the dataset to calculate the mean and std of the stock trading data

# use the helper function to sample
from curatedataset import non_stable_curate_run
import os

# open the path with the json config file and loop through each json file
for json_file in os.listdir(stable_training_json_path):
    if json_file.endswith(".json"):
        json_path = os.path.join(stable_training_json_path, json_file)
              
        non_stable_curate_run(json_path, num_episodes=5, trade_range = [0.05, 0.7])
    

In [ ]:
# read one of the json file
import os
import json
json_file_path = os.listdir(stable_training_json_path)[0]
with open(os.path.join(stable_training_json_path, json_file_path), "r") as f:
    config = json.load(f)

print(config)

# extract the configuration parameters
stock_name = config['stock_name']
start_date = config['start_date']
num_days = config['num_days']
interval = config['interval']
indicators = config['indicators']
init_balance = config['init_balance']
output_path = config['output_path']


In [ ]:
# we can use the curated datasets to calculate the mean and std of the stock trading data using helper function
from curatedataset import calc_meanstd_datasets
dataset_mean_std = calc_meanstd_datasets(output_path,['positive', 'negative', 'neutral'])

In [ ]:
# there are utility functions in curatedataset to help make gymnasium environment from the stock data
from curatedataset import makegymenv
stable_env, obs_space, act_space, col, data = makegymenv(
    stock_name=stock_name, start_date=start_date, period=num_days, interval=interval, 
    indicators=indicators, normalize=dataset_mean_std, init_balance=init_balance)

# Stable-baselines3 library can be used to create and train the agents
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from train_stable_agent import evaluate_stable_agent

num_cpu = 6
# vectorized environment
env_vec = SubprocVecEnv([lambda: stable_env for i in range(num_cpu)])
# create the agent
modelA2C = A2C("MlpPolicy", env_vec, verbose=1)
# we can evaluate the agent before training
mean_reward, std_reward = evaluate_stable_agent(modelA2C, Monitor(stable_env), n_eval_ep=10, deterministic=False)
print(f"Pre-training -> mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

# train the agent
modelA2C.learn(total_timesteps=len(data)*80)

# evaluate the agent after training
mean_reward, std_reward = evaluate_stable_agent(modelA2C, Monitor(stable_env), n_eval_ep=10, deterministic=False)
print(f"Post training -> mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
# the steps above can be used to train multiple agents and store them in a folder with the utility function in get_agent.py
from train_stable_agent import full_run

json_files = os.listdir(stable_training_json_path)

# loop through all json file and train stable-baseline agent with utility function
for file in json_files:
    # check if file ends with json
    if file.endswith(".json"):
        runfile = os.path.join(stable_training_json_path, file)
        print(f"Training agent with {runfile}")
        with open(runfile, "r") as f:
            runconfig = json.load(f)
        output_path = runconfig["output_path"]
        # check if the agent file has already been created
        # create a list of files in the output path and end with .zip

        if os.path.exists(output_path) and len([file for file in os.listdir(output_path) if file.endswith(".zip")]) == 3:
            print(f"Agent with {runfile} already trained")
            continue
        else:
            try:
                full_run(runfile)
            except Exception as e:
                print(f"Error training agent with {runfile}")
                print(e)
                continue

In [ ]:
# now that stable-baseline agent is trained, we can go and sample them in the stock trading environment with different dates

from datetime import datetime, timedelta
# calculate the new start_date as the start_date + num_days for training stable-baseline agent
start_date = datetime.strptime(start_date, "%Y-%m-%d")
new_start_date = start_date + timedelta(days=num_days)
"""
# we want to sample the agents for the days different between start date and now (today) minus 120 days (those 120 days for testing)
today = datetime.today().strftime("%Y-%m-%d")
today = datetime.strptime(today, "%Y-%m-%d")
day_diff = (today - new_start_date).days


if day_diff < 180:
    num_days = day_diff
else:
    num_days = day_diff - 180
"""
num_days = 800

new_start_date = new_start_date.strftime("%Y-%m-%d")
data_output_path = "offline_stock_trade_data/"
data_json_path = "offline_stock_trade_data_config/"

# save these parameter as json config file using the utility function from getsttickers
#create_json_files(all_tickers, new_start_date, num_days, interval, indicators, init_balance, data_output_path, data_json_path)

In [ ]:
# read one of the json file
json_file_path = os.listdir(data_json_path)[0]

with open(os.path.join(data_json_path, json_file_path), "r") as f:
    config = json.load(f)

print(config)

# extract the configuration parameters
stock_name = config['stock_name']
start_date = config['start_date']
num_days = config['num_days']
interval = config['interval']
indicators = config['indicators']
init_balance = config['init_balance']
output_path = config['output_path']

In [ ]:
# import utility for running environment
from curatedataset import makegymenv, run_env

# make the gymnasium environment
data_env, obs_space, act_space, col, data = makegymenv(
    stock_name=stock_name, start_date=start_date, period=num_days, interval=interval, 
    indicators=indicators, normalize=False, init_balance=init_balance, random=True)

In [ ]:
# import Agent class
from get_agent import Agent, TradingAlgorithm

# here is how to run the environment with a random agent
momentum_algo = 'momentum_stoch_rsi'
# find the momentum_stoch_rsi column
momentum_stoch_rsi_col = col.index(momentum_algo)
momentum_trade_algo = TradingAlgorithm(algo_type = momentum_algo, indicator_column = momentum_stoch_rsi_col, amount_range = [0.05,0.4])
# by default agent has safeguard set to true, it prevent agent from buying if the budget is not enough to buy any more shares
momentum_algo_agent = Agent(data_env, 'algo', algo = momentum_trade_algo)
# get the list of date from the data
env_date = data.index.strftime("%Y-%m-%d").tolist()

trade_data = run_env(momentum_algo_agent, stock_name, data_env, num_episodes=3, date=env_date, normalize_param=False, deterministic=False)

In [ ]:
# we will first sample non-stable agents and store their interaction with the environment in a dataset
# this can be used to calculate the mean and std of the stock trading data for stable agents
from curatedataset import non_stable_curate_run
import os

# open the path with the json config file and loop through each json file
for json_file in os.listdir(data_json_path):
    if json_file.endswith(".json"):
        json_path = os.path.join(data_json_path, json_file)
        print(f"Curating data with {json_path}")
        # run the environment with random start date within the period
        with open(json_path, 'r') as f:
            data = json.load(f)
        # get the ticker
        ticker = data['stock_name']
        output = data["output_path"]
        # check if the output path exists and if it contain data, if so skip
        if os.path.exists(output) and len(os.listdir(output)) == 4:
            print(f"Data already curated for {ticker} from non-stable agents")
            continue
        else:
            try:
                non_stable_curate_run(json_path, random = True)
            except Exception as e:
                print(f"Error curating data with {json_path} from non-stable agents")
                print(e)
                continue

In [ ]:
# we can use the curated datasets to calculate the mean and std of the stock trading data using helper function like previously
from curatedataset import calc_meanstd_datasets, makegymenv, run_env
dataset_mean_std = calc_meanstd_datasets(output_path,['positive', 'negative', 'neutral'])

In [ ]:
import os
from get_agent import Agent
# make the gymnasium environment
data_env, obs_space, act_space, col, data = makegymenv(
    stock_name=stock_name, start_date=start_date, period=num_days, interval=interval, 
    indicators=indicators, normalize=dataset_mean_std, init_balance=init_balance, random=True)
# here is how to run the environment with the trained stable-baseline agent
# get the list of date from the data
env_date = data.index.strftime("%Y-%m-%d").tolist()
# get the folder where the trained agents are stored
agent_folder = os.path.join(agent_output_path, stock_name)
# get the a2c agent path
for file in os.listdir(agent_folder):
    if file.endswith(".zip") and "a2c" in file:
        a2c_agent_path = os.path.join(os.getcwd(),agent_folder, file)
        break
a2c_agent = Agent(data_env, "stable-baselines-a2c", model_path = a2c_agent_path)
# run the environment with the agent
trade_data = run_env(a2c_agent, stock_name, data_env, num_episodes=5, date=env_date, normalize_param=dataset_mean_std, deterministic=False)

# inspect the data keys
print(data.keys())


In [ ]:
# the aboved step can be done with utility function from curatedataset.py
from curatedataset import stable_curate_run

data_json_files = os.listdir(data_json_path)

# loop through all json file and run environment with trained stable agent and other algorithms
for file in data_json_files:
    # check if file ends with json
    if file.endswith(".json"):
        runfile = os.path.join(data_json_path, file)
        print(f"Curating data with {runfile}")
        # open the json file and read the content
        with open(runfile, 'r') as f:
            data = json.load(f)
        # get the ticker
        ticker = data['stock_name']
        output = data["output_path"]
        # check if the output path exists and if it contain data, if so skip
        if os.path.exists(output) and len(os.listdir(output)) == 7:
            print(f"Data already curated for {ticker} from stable agents")
            continue
        else:
            try:
                # joining the output path with ticker
                trained_stable_agent_path = os.path.join(agent_output_path, ticker)
                stable_curate_run(runfile, trained_stable_agent_path, num_episodes=80)
            except Exception as e:
                print(f"Error curating data with {runfile} from stable agents")
                print(e)
                continue
